# SQL DATA ANALYSIS TASK

# Context
Fenix is a company that is in renewable energy. To this end, it sells solar kits to populations who are in need. Customers have the option of paying for the kit in cash or on credit. If the customer chooses to buy on credit, he can choose between one year (360 days) or two years (720). He is then asked to pay an amount per day (daily_rate), this amount depending on the duration of the loan and the type of kit he has chosen and corresponds to a day of light. So every time he pays that amount, he has a day of light. After 24 hours the system hangs and it has to make another payment to take advantage of the light. He can decide to pay for more than one day of light. The day number is then the whole part of the amount / daily rate ratio. After this number of days, the system still locks. When a customer spends more than 30 days without light, he is considered inactive. Paid vs Expected (PvE) means the ratio between the total amount paid by the client and the amount he should have paid up to date. It allows to appreciate the recovery.

Let's consider the following tables

Loan_current_daitails

loan_id	|customer_id|daily_rate|days_elapsed|Days_til_lock_out|Total_paid
--------|-----------|----------|------------|-----------------|----------
loan_id_1|customer_id_1|daily_rate_1|days_elapsed_1|Days_til_lock_out_1|Total_paid_1
loan_id_2|customer_id_2|daily_rate_2|days_elapsed_2|Days_til_lock_out_2|Total_paid_2
.........|.............|............|..............|...................|............
loan_id_n|customer_id_n	|daily_rate_n|days_elapsed_n|Days_til_lock_out_n|Total_paid_n


Person_democraphic

customer_id	|country|	region|	profession
------------|-------|---------|-----------
customer_id_1|	Country_1	|Region_1	|profession_1
.............|	............|..........	|............
customer_id_j|	Country_j|	Region_j	|profession_j


Sales_details

loan_id	|Product_type	|Date_fullfiled	|loan_duration
--------|----------------|--------------|-------------
loan_id_1	|Product_type_1	|Date_fullfiled_1	|loan_duration_1
...|	...	|...	|...
loan_id_n|	Product_type_n|	Date_fullfiled_n|	loan_duration_n

### Tables description

* **Loan_id:** unique loan identifier

* **Customer_id:** unique customer identifier

* **Days_elapsed:** the number of days the client spends with us

* **Daily_rate:** expected amount from customer per day

* **Days_til_lock_out:** If positive, then corresponds to the number of days of light remaining before the system hangs. If negative, corresponds to the number of days since the system was blocked (the client has been without light since this number of days)

* **Total_paid:** total amount to be paid by the customer

* **Country:** Customer's country

* **Region:** customer department

* **Profession:** client's profession

* **Product_type:** the type of kit purchased by the customer

* **Loan_duration:** the duration of the customer's loan (360 or 720 days)

* **Date_fullfiled:** the date of the sale

### The datasets
The datasets exist in csv formats and can be downloaded and saved on disk

In [3]:
!pip install ipython-sql
%load_ext sql
%sql sqlite:///testdb.sqlite

  Running setup.py bdist_wheel for prettytable: started
  Running setup.py bdist_wheel for prettytable: finished with status 'done'
  Stored in directory: C:\Users\OWNER\AppData\Local\pip\Cache\wheels\80\34\1c\3967380d9676d162cb59513bd9dc862d0584e045a162095606
Successfully built prettytable


In [4]:
%%sql
DROP TABLE IF EXISTS Loan_current_daitails;
DROP TABLE IF EXISTS Person_democraphic;
DROP TABLE IF EXISTS Sales_details;

 * sqlite:///testdb.sqlite
Done.
Done.
Done.


[]

In [69]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("testdb.sqlite")
customers = pd.read_csv('Loan_current_daitails.csv')
orders = pd.read_csv('Person_democraphic.csv')
salesman = pd.read_csv('Sales_details.csv')
orders.rename(columns={'Profession ': 'Profession'}, inplace=True)

customers.to_sql("Loan_current_daitails", conn, index=False, if_exists="replace")
orders.to_sql("Person_democraphic", conn, index=False, if_exists="replace")
salesman.to_sql("Sales_details", conn, index=False, if_exists="replace")

### Write a SQL code to determine the total number of loans on the one hand and the total number of customers on the other

In [70]:
%%sql

SELECT COUNT(DISTINCT loan_id)
FROM Loan_current_daitails as column_name_count;

 * sqlite:///testdb.sqlite
Done.


COUNT(DISTINCT loan_id)
5000


In [71]:
%%sql
SELECT COUNT(DISTINCT customer_id)
FROM Loan_current_daitails as column_name_count;

 * sqlite:///testdb.sqlite
Done.


COUNT(DISTINCT customer_id)
4979


### Write a SQL code to determine the total number of loans by type of kit

In [72]:
%%sql
SELECT product_type, COUNT(DISTINCT(loan_id)) FROM Sales_details
GROUP BY product_type

 * sqlite:///testdb.sqlite
Done.


product_type,COUNT(DISTINCT(loan_id))
Fenix 2,841
Fenix 3,974
Fenix Power 2+ Tondeuse,18
Fenix Power 4 + Woofer BJ,35
Fenix Power TV 19in FTA BJ,14
"Fenix Power TV 19in FTA Upgrade, BJ",28
Fenix Power TV 24in FTA BJ,634
"Fenix Power TV 24in FTA Upgrade, BJ",177
Fenix Radio +3,1791
Fenix Radio +5,422


### Write SQL code to determine the number of inactive loans

In [73]:
%%sql

SELECT COUNT(DISTINCT(loan_id)) FROM Loan_current_daitails
WHERE days_til_lockout < 0

 * sqlite:///testdb.sqlite
Done.


COUNT(DISTINCT(loan_id))
1294


### Write a SQL code to determine the regions where the total amount of payments exceeds 1,000,000

In [74]:
%%sql
SELECT [Person_democraphic].region
  FROM [Person_democraphic] INNER JOIN Loan_current_daitails 
    ON [Person_democraphic].customer_id = Loan_current_daitails.customer_id
WHERE Loan_current_daitails.total_paid > 1000000

 * sqlite:///testdb.sqlite
Done.


region


### Write a SQL code to classify by region and in ascending order, the loans according to the total amount paid

In [75]:
%%sql
SELECT [Person_democraphic].region, Loan_current_daitails.loan_id, Loan_current_daitails.total_paid
  FROM [Person_democraphic] INNER JOIN Loan_current_daitails 
    ON [Person_democraphic].customer_id = Loan_current_daitails.customer_id
GROUP BY [Person_democraphic].region 
ORDER BY Loan_current_daitails.total_paid ASC

 * sqlite:///testdb.sqlite
Done.


region,loan_id,total_paid
Zou,891510,3000
Mono,798485,11400
Atlantique,552403,13825
Atacora,546198,14160
Alibori,586196,24160
Plateau,653030,32325
Collines,617665,35970
Borgou,565181,44200
Couffo,447804,51500
Donga,560165,51700



### Write a SQL code to determine the top 10 occupations by region with the highest total amount of payments

In [76]:
%%sql
SELECT [Person_democraphic].Profession , [Person_democraphic].region, Loan_current_daitails.total_paid
  FROM [Person_democraphic] INNER JOIN Loan_current_daitails 
    ON [Person_democraphic].customer_id = Loan_current_daitails.customer_id
GROUP BY [Person_democraphic].region 
ORDER BY Loan_current_daitails.total_paid DESC
LIMIT 10;

 * sqlite:///testdb.sqlite
Done.


Profession,region,total_paid
Statistian,Littoral,61000
Tehnician,Oueme,52355
Teacher,Donga,51700
Cmmercial,Couffo,51500
Ingenieur,Borgou,44200
Tehnician,Collines,35970
Nurse,Plateau,32325
Doctor,Alibori,24160
Farmer,Atacora,14160
Tehnician,Atlantique,13825
